<a href="https://colab.research.google.com/github/advik-7/Audio_pre_processing-effect-on-ASR/blob/main/Side_Index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!pip install sentence-transformers faiss-gpu openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: h11
    Found existing installation: h11 0.9.0
    Uninstalling h11-0.9.0:
      Successfully uninstalled h11-0.9.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 0.9.1
    Uninstalling httpcore-0.9.1:
      Successfully uninstalled httpcore-0.9.1
  Attempting uninstall: httpx
    Found existing installation: httpx 0.13.3
    Uninstalling httpx-0.13.3:
      Successfully uninstalled httpx-0.13.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googletrans 3.1.0a0 requires httpx==0.13.3, but you have httpx 0.27.2 which is incompatible.


In [30]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer

In [31]:
def load_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        paragraphs = [line.strip() for line in file if line.strip()]  # Skip blank lines
    return paragraphs

In [32]:
# Load the paragraphs from your text file
file_path = "./complete_text.txt"
text_corpus = load_text_file(file_path)

In [33]:
# Optionally split long paragraphs into smaller chunks
def split_into_chunks(paragraphs, chunk_size=200):
    chunks = []
    for paragraph in paragraphs:
        words = paragraph.split()
        chunks.extend([" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)])
    return chunks

chunks = split_into_chunks(text_corpus)

In [34]:
print(f"Total chunks: {len(chunks)}")

Total chunks: 254


In [35]:
import faiss
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings = model.encode(chunks)
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)
faiss_index_path = "/content/faiss_index_chunks.bin"
faiss.write_index(index, faiss_index_path)

print(f"FAISS index saved at: {faiss_index_path}")

chunks_file_path = "/content/faiss_index_chunks.txt"
with open(chunks_file_path, 'w', encoding='utf-8') as file:
    for chunk in chunks:
        file.write(chunk + "\n")

print(f"Chunks saved at: {chunks_file_path}")

metadata_file_path = "/content/faiss_index_metadata.txt"
with open(metadata_file_path, 'w', encoding='utf-8') as meta_file:
    for idx, chunk in enumerate(chunks):
        meta_file.write(f"Vector Index: {idx} -> Chunk: {chunk}\n")

print(f"Metadata saved at: {metadata_file_path}")


FAISS index saved at: /content/faiss_index_chunks.bin
Chunks saved at: /content/faiss_index_chunks.txt
Metadata saved at: /content/faiss_index_metadata.txt


In [36]:
def retrieve_relevant_chunks(query, top_k=5):
    query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, top_k)
    return [chunks[i] for i in indices[0]]

# Example query
query = "what are the problems faced by farmers"
relevant_chunks = retrieve_relevant_chunks(query)

print("Relevant Chunks:", relevant_chunks)


Relevant Chunks: ['Farmers are under significant financial pressure due to increasing costs of agricultural tools, pesticides, and fertilizers. The economic burden, coupled with low returns from the market, drives many farmers into debt. The government is urged to support them, especially in terms of education and access to more sustainable farming practices.', 'Challenges such as drought, the need for more fertilizers, and the struggle for proper compensation are mentioned. Despite these challenges, many speakers remain optimistic and encourage other farmers to join the movement.', 'It emphasizes the role of society and the government in supporting the farmers. Farmers are seen as the backbone of agriculture, but the entire system, including the education system, policy, and market access, must be improved to ensure they are not left behind in terms of knowledge or financial stability.', 'Farmers are experiencing financial hardship due to the theft of their sandalwood. Despite years o

In [37]:
import google.generativeai as genai

In [38]:
from google.colab import userdata

In [46]:
genai.configure(api_key="")
model = genai.GenerativeModel("gemini-1.5-flash")

def query_llm(prompt, context):
    """
    Modify the prompt for the LLM to act as an AI agent for Kannada farmers' knowledge.
    """
    complete_prompt = (
        f"{context}\n\n"
        f"ನೀವು ಕರ್ನಾಟಕದ ರೈತರಿಗೆ ಕೃಷಿ ಬಗ್ಗೆ ಜ್ಞಾನ ಹರಡುವ ಸಲಹೆಗಾರರಾಗಿ ಕಾರ್ಯನಿರ್ವಹಿಸುತ್ತಿದ್ದೀರಿ. "
        f"ನೀವು ರೈತರನ್ನು ಕಷ್ಟಕರ ಪರಿಸ್ಥಿತಿಗಳಲ್ಲಿ ಮಾರ್ಗದರ್ಶನ ನೀಡುತ್ತೀರಿ ಮತ್ತು ಸಂಕ್ಷಿಪ್ತವಾಗಿ, ಸರಳವಾಗಿ, "
        f"ಅವರ ಸಮಸ್ಯೆಗಳನ್ನು ಸुलಭವಾಗಿ ವಿವರಿಸಿ ಪರಿಹಾರಗಳನ್ನು ನೀಡುತ್ತೀರಿ. "
        f"ನೀವು ಹೇಳುವ ಪ್ರತಿಯೊಂದು ಉತ್ತರವೂ ಉಪಯುಕ್ತ, ಪ್ರಾಯೋಗಿಕ ಮತ್ತು ರೈತರ ಸಹಾಯಕ್ಕಾಗಿ ಮಾತ್ರ ಆಗಿರುತ್ತದೆ."
        f"\n\n"
        f"ಪ್ರಶ್ನೆ: {prompt}\n"
        f"ಉತ್ತರ: "
        f"ನೀವು ರೈತರಿಗೆ ಸಹಾಯ ಮಾಡುವಂತೆ, ಸಿದ್ಧಾಂತಗಳನ್ನು ಸರಳವಾಗಿ ಹಾಗೂ ಗಮ್ಯವಾಗಿಯೂ ವಿವರಿಸಿ, "
        f"ಅವರ ಸಮಸ್ಯೆಗಳಿಗೆ ಪರಿಹಾರವನ್ನು ಕಠಿಣವಾಗಿ, ಆದರೆ ಶ್ರದ್ಧೆಯಿಂದ ಪ್ರಸ್ತುತಪಡಿಸಿ."
    )
    response = model.generate_content([complete_prompt])
    return response.text



In [40]:
!pip install googletrans==3.1.0a0

  Using cached httpx-0.13.3-py3-none-any.whl.metadata (25 kB)
  Using cached httpcore-0.9.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached h11-0.9.0-py2.py3-none-any.whl.metadata (8.1 kB)
Using cached httpx-0.13.3-py3-none-any.whl (55 kB)
Using cached httpcore-0.9.1-py3-none-any.whl (42 kB)
Using cached h11-0.9.0-py2.py3-none-any.whl (53 kB)
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Successfully uninstalled h11-0.14.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 1.0.7
    Uninstalling httpcore-1.0.7:
      Successfully uninstalled httpcore-1.0.7
  Attempting uninstall: httpx
    Found existing installation: httpx 0.27.2
    Uninstalling httpx-0.27.2:
      Successfully uninstalled httpx-0.27.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langsmith 0.1.142 requir

In [41]:
from googletrans import Translator

In [42]:
translator = Translator()
query_kn = "ರೈತರು ಎದುರಿಸುತ್ತಿರುವ ಸಮಸ್ಯೆಗಳೇನು"
query_en = translator.translate(query_kn, src="kn", dest="en")

In [47]:
context = "\n".join(relevant_chunks)
prompt = query_kn
answer = query_llm(prompt, context)

print("Answer:", answer)

Answer: ನಮಸ್ಕಾರ ರೈತ ಬಂಧುಗಳೇ,

ನಾನು ನಿಮ್ಮ ಜೊತೆ ಕೆಲಸ ಮಾಡಲು ಬಂದಿದ್ದೇನೆ. ನೀವು ಎದುರಿಸುತ್ತಿರುವ ಸಮಸ್ಯೆಗಳು ನನಗೆ ತಿಳಿದಿವೆ.  ಅವುಗಳನ್ನು ಸರಳವಾಗಿ ಹೇಳುವುದಾದರೆ:

**1. ಹೆಚ್ಚುತ್ತಿರುವ ವೆಚ್ಚಗಳು:**  ಕೃಷಿ ಉಪಕರಣಗಳು, ರಸಗೊಬ್ಬರ, ಮತ್ತು ಕೀಟನಾಶಕಗಳ ಬೆಲೆಗಳು ಹೆಚ್ಚಾಗುತ್ತಿವೆ.  ಇದರಿಂದ ನಿಮ್ಮ ಲಾಭ ಕಡಿಮೆಯಾಗುತ್ತಿದೆ.

**ಪರಿಹಾರ:**

* **ಸಾವಯವ ಕೃಷಿ:** ರಾಸಾಯನಿಕ ರಸಗೊಬ್ಬರ ಮತ್ತು ಕೀಟನಾಶಕಗಳ ಬಳಕೆ ಕಡಿಮೆ ಮಾಡಿ, ಸಾವಯವ ವಿಧಾನಗಳನ್ನು ಅಳವಡಿಸಿಕೊಳ್ಳಿ. ಇದರಿಂದ ದೀರ್ಘಕಾಲದಲ್ಲಿ ವೆಚ್ಚ ಕಡಿಮೆಯಾಗುತ್ತದೆ ಮತ್ತು ಉತ್ಪನ್ನದ ಗುಣಮಟ್ಟ ಹೆಚ್ಚಾಗುತ್ತದೆ.  ಸರ್ಕಾರದಿಂದ ಸಾವಯವ ಕೃಷಿಗೆ ಪ್ರೋತ್ಸಾಹ ಯೋಜನೆಗಳಿವೆ, ಅವುಗಳ ಬಗ್ಗೆ ಮಾಹಿತಿ ಪಡೆಯಿರಿ.
* **ಸಹಕಾರಿ ಸಂಘಗಳು:** ಇತರ ರೈತರೊಂದಿಗೆ ಸೇರಿ ಸಹಕಾರಿ ಸಂಘಗಳನ್ನು ರಚಿಸಿ. ಇದರಿಂದ ಉಪಕರಣಗಳು ಮತ್ತು ಇನ್ಪುಟ್ಸ್ ಅನ್ನು ಒಟ್ಟಾಗಿ ಖರೀದಿಸುವುದರಿಂದ ಬೆಲೆ ಕಡಿಮೆಯಾಗುತ್ತದೆ.


**2. ಕಡಿಮೆ ಮಾರುಕಟ್ಟೆ ಬೆಲೆ:** ನಿಮ್ಮ ಉತ್ಪನ್ನಗಳಿಗೆ ಸರಿಯಾದ ಬೆಲೆ ಸಿಗುತ್ತಿಲ್ಲ.

**ಪರಿಹಾರ:**

* **ಮಾರುಕಟ್ಟೆ ಜ್ಞಾನ:**  ನಿಮ್ಮ ಉತ್ಪನ್ನಗಳಿಗೆ ಉತ್ತಮ ಬೆಲೆ ಸಿಗುವ ಮಾರುಕಟ್ಟೆಗಳನ್ನು ಗುರುತಿಸಿ.  ಸಂಬಂಧಿತ ಮಾಹಿತಿ ಪಡೆಯಲು ಸರ್ಕಾರದ ಕೃಷಿ ಇಲಾಖೆ ಅಥವಾ ಸ್ಥಳೀಯ ಮಾರುಕಟ್ಟೆ ಸಮಿತಿಗಳನ್ನು ಸಂಪರ್ಕಿಸಿ.
* **ಮೌಲ್ಯವರ್ಧನೆ:** ನಿಮ್ಮ ಉತ್ಪನ್ನಗಳನ್ನು ಮೌ